In [1]:
import torch
from torchvision import datasets, transforms
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
# get GPU info
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)

In [3]:
torch.cuda.is_available()

True

In [4]:
# get number of GPUs available: useful if you need to count them for dataparallel
torch.cuda.device_count()

1

In [5]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [6]:
# get memory usage: similar to gpustat in CLI
!pip install pynvml
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 4.8 MB/s 
total    : 15843721216
free     : 15840575488
used     : 3145728


In [7]:
## import libs and set device

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


device = torch.device("cuda")
#device = torch.device("cpu")


In [8]:
# Set params and download everything


# Training settings: in a py script could set these in cli with argparse
batch_size=64
test_batch_size=1000
epochs=1
lr = 1.0
gamma = 0.7
log_interval = 10    # how many batches to wait before logging training status

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}


# add cuda kwargs
cuda_kwargs = {'num_workers': 1,
                'pin_memory': True,
                'shuffle': True}
train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)


transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)


train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [9]:
device

device(type='cuda')

In [10]:
%%time

print(f'model is on device: {next(model.parameters()).device}')

## train
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()



model is on device: cuda:0
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297760
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.158647
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.768124
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.730324
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.377267
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.261988
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.235956
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.260014
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.243189
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.322156
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.215591
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.291029
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.241238
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.118587
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.234712
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.229434
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.439597
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.174933
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.099117
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.091087
Tr

In [11]:
# save results to current directory
torch.save(model.state_dict(), "mnist_cnn.pt")
